## Kombinasi 5 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier handling with log tranformation
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("../UFC_train.csv")

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [3]:
# Delete duplicate
print(f"Shape before dropping duplicate : {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate : {df.shape}")

Shape before dropping duplicate : (5410, 144)
Shape after dropping duplicate : (5410, 144)


In [4]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [5]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [6]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [7]:
# Impute missing value using iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import ExtraTreeRegressor

imputer = IterativeImputer(random_state=42, max_iter=15, initial_strategy='median', estimator=ExtraTreeRegressor())
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)

/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [8]:
# Drop R_fighter, B_fighter, Referee, location, date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [9]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.000000,0.500000,0.553125,0.550000,0.003662,0.280000,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.000000,0.500000,0.477500,0.302500,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,185.42,195.58,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,1.0,5.0,6.0,3.0,0.0,177.80,185.42,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,1.0,160.02,167.64,135.0,34.0,31.0


In [10]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
B_Stance     64  0.011830


In [11]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

In [12]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [13]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [14]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [15]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 461
B_avg_opp_KD             633
B_avg_SIG_STR_pct        149
B_avg_opp_SIG_STR_pct    121
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               81
R_Weight_lbs             302
B_age                     32
R_age                     74
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 8.521256931608132%
B_avg_opp_KD = 11.700554528650647%
B_avg_SIG_STR_pct = 2.754158964879852%
B_avg_opp_SIG_STR_pct = 2.236598890942699%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 3.4380776340110906%
B_avg_opp_SUB_ATT = 6.617375231053605%
B_avg_REV = 10.036968576709796%
B_avg_opp_REV = 9.870609981515711%
B_avg_SIG_STR_att = 3.4565619223659887%
B_avg_SIG_STR_landed = 3.253234750462107%
B_avg_opp_SIG_STR_att = 3.4195933456561924%
B_avg_opp_SIG_STR_landed = 2.865064695009242%
B_avg_TOTAL_STR_att = 2.2735674676524953%
B_avg_TOTAL_STR_landed = 2.0702402957486

In [16]:
# Add a constant value to make all values positive
df_number_positive = df_number + abs(df_number.min()) + 1

df_number_log = np.log(df_number_positive)


In [17]:
df_number_log.head()

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,0.000000,0.405465,0.440269,0.438255,0.003655,0.246860,0.015504,0.000000,0.405465,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.817944,5.802722,5.707110,3.806662,3.713572
1,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,0.124703,0.000000,0.000000,...,0.000000,1.609438,0.000000,0.693147,0.000000,5.810362,5.832938,5.655992,3.828641,3.912023
2,0.000000,0.405465,0.390351,0.264285,0.693147,0.693147,1.098612,0.295464,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.825469,5.855015,5.655992,3.850148,3.931826
3,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,0.060625,0.000000,0.000000,...,0.693147,1.791759,1.945910,1.386294,0.000000,5.802722,5.825469,5.655992,3.850148,3.988984
4,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,0.000244,0.117783,0.405465,...,0.000000,1.098612,1.386294,0.000000,0.693147,5.747544,5.771566,5.525453,3.970292,3.931826


In [18]:
df = pd.concat([df_object, df_number_log], axis=1)

In [19]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 461
B_avg_opp_KD             633
B_avg_SIG_STR_pct        149
B_avg_opp_SIG_STR_pct    121
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               81
R_Weight_lbs             302
B_age                     32
R_age                     74
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 8.521256931608132%
B_avg_opp_KD = 11.700554528650647%
B_avg_SIG_STR_pct = 2.754158964879852%
B_avg_opp_SIG_STR_pct = 2.236598890942699%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 3.4380776340110906%
B_avg_opp_SUB_ATT = 6.617375231053605%
B_avg_REV = 10.036968576709796%
B_avg_opp_REV = 9.870609981515711%
B_avg_SIG_STR_att = 3.4565619223659887%
B_avg_SIG_STR_landed = 3.253234750462107%
B_avg_opp_SIG_STR_att = 3.4195933456561924%
B_avg_opp_SIG_STR_landed = 2.865064695009242%
B_avg_TOTAL_STR_att = 2.2735674676524953%
B_avg_TOTAL_STR_landed = 2.0702402957486

In [20]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [21]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Middleweight,Orthodox,Southpaw,Red,0.000000,0.405465,0.440269,0.438255,0.003655,0.246860,...,0.000000,0.000000,0.000000,0.000000,5.817944,5.802722,5.707110,3.806662,3.713572,male
1,Lightweight,Orthodox,Orthodox,Blue,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,...,1.609438,0.000000,0.693147,0.000000,5.810362,5.832938,5.655992,3.828641,3.912023,male
2,Welterweight,Orthodox,Orthodox,Blue,0.000000,0.405465,0.390351,0.264285,0.693147,0.693147,...,0.000000,0.000000,0.000000,0.000000,5.825469,5.855015,5.655992,3.850148,3.931826,male
3,Welterweight,Southpaw,Orthodox,Blue,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,...,1.791759,1.945910,1.386294,0.000000,5.802722,5.825469,5.655992,3.850148,3.988984,male
4,Bantamweight,Orthodox,Orthodox,Red,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,...,1.098612,1.386294,0.000000,0.693147,5.747544,5.771566,5.525453,3.970292,3.931826,male


In [22]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [23]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [24]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [25]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [26]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)

In [27]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,7,Orthodox,Southpaw,Red,0.000000,0.405465,0.440269,0.438255,0.003655,0.246860,...,0.000000,0.000000,0.000000,0.000000,5.817944,5.802722,5.707110,3.806662,3.713572,1
1,5,Orthodox,Orthodox,Blue,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,...,1.609438,0.000000,0.693147,0.000000,5.810362,5.832938,5.655992,3.828641,3.912023,1
2,6,Orthodox,Orthodox,Blue,0.000000,0.405465,0.390351,0.264285,0.693147,0.693147,...,0.000000,0.000000,0.000000,0.000000,5.825469,5.855015,5.655992,3.850148,3.931826,1
3,6,Southpaw,Orthodox,Blue,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,...,1.791759,1.945910,1.386294,0.000000,5.802722,5.825469,5.655992,3.850148,3.988984,1
4,3,Orthodox,Orthodox,Red,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,...,1.098612,1.386294,0.000000,0.693147,5.747544,5.771566,5.525453,3.970292,3.931826,1


In [28]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
0,7,0.000000,0.405465,0.440269,0.438255,0.003655,0.246860,0.015504,0.000000,0.405465,...,0,0,0,0,0,1,0,0,0,1
1,5,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,0.124703,0.000000,...,0,0,0,1,0,0,0,1,0,0
2,6,0.000000,0.405465,0.390351,0.264285,0.693147,0.693147,1.098612,0.295464,0.000000,...,0,0,0,1,0,0,0,1,0,0
3,6,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,0.060625,0.000000,...,1,0,0,1,0,0,0,1,0,0
4,3,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,0.000244,0.117783,...,0,0,0,1,0,0,0,0,0,1


In [29]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# Feature selection with Lasso
X = df.drop('B_Reach_cms', axis=1)
y = df['B_Reach_cms']

# Create the KNN model
lasso = Lasso(random_state=42)

lasso.fit(X, y)

# Get features with non-zero weights
non_zero_features = np.where(lasso.coef_ != 0)[0]

# Display the feature indices with non-zero weights
print("Features with non-zero weights:", non_zero_features)

Features with non-zero weights: []


In [30]:
# Define the parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
    'fit_intercept': [True, False],
    'precompute': [True, False],
    'max_iter': [1000, 2000, 5000],
    'tol': [0.0001, 0.001, 0.01],
    'selection': ['cyclic', 'random']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the data to perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)

Best Parameters: {'alpha': 0.1, 'fit_intercept': True, 'max_iter': 1000, 'precompute': True, 'selection': 'cyclic', 'tol': 0.0001}
Best R-squared Score: -0.00026876949530802997


In [31]:
# Feature importance
feature_importance = grid_search.best_estimator_.coef_
print(feature_importance)

# Get the indices of the feature importance
feature_importance_indices = np.argsort(feature_importance)

[ 0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0.  0.  0.  0.  0. -0. -0. -0.  0.  0.  0. -0.
  0.  0.  0.  0.  0. -0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0.  0.  0. -0.  0.  0.  0.  0.  0. -0. -0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0. -0.  0. -0.  0.  0.
 -0. -0.  0.  0.]


In [32]:
print(len(feature_importance_indices))

148


In [33]:
non_zero_features = np.where(feature_importance != 0)[0]

non_zero_features_indices = np.argsort(feature_importance[non_zero_features])

print(len(non_zero_features_indices))

0


In [34]:
# df_feature_selection is df that has column with non-zero weight
df_feature_selection = df.iloc[:, non_zero_features]

df_feature_selection.head()

""
0
1
2
3
4


In [35]:
df_feature_selection['B_Reach_cms'] = y

/var/folders/53/kv053qr13vxg2_y_1lg6hjg80000gn/T/ipykernel_95211/3603037234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature_selection['B_Reach_cms'] = y


In [36]:
# Export to CSV
df_feature_selection.to_csv('../Punya Andi/UFC_kombinasi5_FS_lasso.csv', index=False)
